<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/File_Share_3_Finding_the_arrival_prediction_and_summary_metrics_of_selected_stop_points_for_a_given_lineID_and_direction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code Progression: Enter line ID, direction. Gives the origination and destination of the lineID. Writes the  stop points between origination and destination in the sequence as they appears in the given direction of travel. Asks the user to enter the no of QSI points and the row number corresponding to them from the csv file. Returns the arrival predictions and summary metrics of the given QSI points.

In [ ]:
# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import csv

# Function to convert time to BST
def convert_to_bst(dt):
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')
    utc_dt = utc_tz.localize(dt)
    bst_dt = utc_dt.astimezone(bst_tz)
    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

# Fetch route details
def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {'serviceTypes': 'Regular'}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        route_sections = data.get('routeSections', [])
        found_direction = False
        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')
                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)
                if all_stop_details:
                    write_to_csv(all_stop_details)
                    display_data_from_csv()
        if not found_direction:
            print(f"No route found for direction '{direction}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Fetch and append stop points
def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        line_strings = data.get('lineStrings', [])
        if line_strings:
            printed_stop_ids = set()
            for line_string in line_strings:
                coordinates = eval(line_string)
                for lon, lat in coordinates[0]:
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)
        else:
            print("No stop points found for this route.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")
    return all_stop_details

# Fetch and append stop details
def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"
    try:
        for attempt in range(retries):
            response = requests.get(url)
            if response.status_code == 429:
                time.sleep(2**attempt)
                continue
            response.raise_for_status()
            data = response.json()
            stop_points = data.get('stopPoints', [])
            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')
                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })
                        printed_stop_ids.add(stop_id)
            else:
                continue
            break
        else:
            pass
    except requests.exceptions.RequestException as e:
        pass

# Write to CSV
def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']
    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)
    print(f"Data written to 'stop_details.csv' successfully.")

# Display data from CSV
def display_data_from_csv():
    print("Data from CSV:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        headers_printed = False
        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Get QSI stop points
def get_qsi_stop_points():
    count_of_qsi_stop_points = int(input("Enter the number of QSI stop points: "))
    qsi_stop_points = []
    for i in range(count_of_qsi_stop_points):
        row_no = int(input(f"Enter the row number for QSI stop point {i+1}: "))
        qsi_stop_points.append(row_no)
    qsi_stop_ids = []
    print("QSI Stop Points:")
    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if int(row['Sl. No.']) in qsi_stop_points:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                qsi_stop_ids.append(row['Stop ID'])
    return qsi_stop_ids

# Main loop to fetch and display data
def main_loop(line_id, direction, qsi_stop_ids):
    nbph = 6  # Fixed number of buses per hour for all QSI stop points
    while True:
        qsi_data = []  # List to accumulate data for all QSI stop points
        for idx, stop_point_id in enumerate(qsi_stop_ids):
            df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)
            if df is not None and station_name is not None:
                total_wawt = df['WAWT'].sum()
                min_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').min().replace(second=0, microsecond=0)
                max_arrival = pd.to_datetime(df['Expected Arrival (BST)'], format='%H:%M:%S').max().replace(second=0, microsecond=0)
                time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60
                awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
                swt = round(60 / (nbph * 2), 2)
                ewt = round(awt - swt, 2)
                summary_df = pd.DataFrame({
                    'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
                    'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
                })
                qsi_data.append({
                    'QSI Stop Point': f"{idx + 1} - {station_name}",
                    'Arrival Predictions': df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']].to_string(index=False),
                    'Summary Metrics': summary_df.to_string(index=False)
                })
            else:
                qsi_data.append({
                    'QSI Stop Point': f"{idx + 1} - Fetch Error",
                    'Arrival Predictions': "No data available",
                    'Summary Metrics': "No data available"
                })

        clear_output(wait=True)
        for qsi in qsi_data:
            print(f"QSI Stop Point: {qsi['QSI Stop Point']}")
            print("Arrival Predictions:")
            print(qsi['Arrival Predictions'])
            print("\nSummary Metrics:")
            print(qsi['Summary Metrics'])
            print("\n--------------------------------------------\n")

        time.sleep(15)

if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")
    fetch_route_details(line_id, direction)
    qsi_stop_ids = get_qsi_stop_points()
    main_loop(line_id, direction, qsi_stop_ids)


QSI Stop Point: 1 - Moorfields Eye Hospital
Arrival Predictions:
Line Vehicle ID Stop Point Direction Expected Arrival (BST)  Headway (minutes)  AWT/bus (minutes)  WAWT
 214    LA19KBP 490010012N   inbound               12:30:00                0.0                0.0   0.0
 214    LA19KBY 490010012N   inbound               12:36:00                6.0                3.0  18.0
 214    LA19KBJ 490010012N   inbound               12:43:00                7.0                3.5  24.5
 214    LA19KAO 490010012N   inbound               12:51:00                8.0                4.0  32.0
 214    LA19KAE 490010012N   inbound               12:58:00                7.0                3.5  24.5

Summary Metrics:
                                                       Metric  Value
                    Number of buses scheduled per hour (nbph)   6.00
                                         Total WAWT (minutes)  99.00
Time difference between 1st and last observed buses (minutes)  28.00
                 